Unzip the Train Data

In [ ]:
! unzip -qq Train_data.zip

Import All The Packages

In [69]:
from keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.models import Model
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img

In [70]:
img_gen = ImageDataGenerator(rotation_range=10, shear_range=.2, zoom_range=.1)
gen = img_gen.flow_from_directory("Train_data")
x_batch, y_batch = gen.next()
x_batch.shape

Found 406 images belonging to 4 classes.


(32, 256, 256, 3)

In [71]:
from keras.applications import ResNet50

In [72]:
res = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3), classes=4)

/home/ankit/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [73]:
for layer in res.layers:
  layer.trainable = False

In [74]:
res.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [75]:
flat = Flatten()(res.output)
d1 = Dense(400, activation="tanh")(flat)
d2 = Dense(100, activation="tanh")(d1)
drop = Dropout(rate=0.2)(d2)
d3 = Dense(4, activation="softmax")(drop)

In [76]:
model = Model(res.input, d3)

model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])

In [12]:
model.fit_generator(gen, steps_per_epoch=5, epochs=5)

Epoch 1/5


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


5/5 [==============================] - 132s 26s/step - loss: 1.0944 - accuracy: 0.5500
Epoch 2/5
5/5 [==============================] - 43s 9s/step - loss: 0.6156 - accuracy: 0.7733
Epoch 3/5
5/5 [==============================] - 50s 10s/step - loss: 0.5094 - accuracy: 0.7750
Epoch 4/5
5/5 [==============================] - 48s 10s/step - loss: 0.2985 - accuracy: 0.8938
Epoch 5/5
5/5 [==============================] - 46s 9s/step - loss: 0.2122 - accuracy: 0.9533


In [13]:
model.evaluate(x_batch, y_batch)

32/32 [==============================] - 18s 569ms/step


[0.21904855966567993, 0.9375]

Upload The Image Or Capture the Image.

In [66]:
cap=cv2.VideoCapture(0)
while True:
   ret, frame= cap.read()
   if ret :
       cv2.imshow("My First cam",frame)

   key=cv2.waitKey(1)
   if ord("q")==key&0xff:
       break
   if ord("c") == key & 0xff:
       cv2.imwrite("1.png",frame)
cap.release()
cv2.destroyAllWindows()

In [66]:
img = load_img("1.png", target_size=(256, 256))
np_img = np.array(img)
a=model.predict(np.array([np_img])).argmax(axis=1)
#gen.class_indices
if a==0:
    print ("The Bulldog, also known as the British Bulldog or English Bulldog, is a medium-sized dog breed. It is a muscular, hefty dog with a wrinkled face and a distinctive pushed-in nose.")
if a==1:
    print ("The Dobermann or Doberman Pinscher in the United States and Canada, is a medium-large breed of domestic dog")
if a==2:
    print ("The German Shepherd is a breed of medium to large-sized working dog that originated in Germany. In the English language, the breed's officially recognized name is German Shepherd Dog")
if a==3:
    print ("The Labrador Retriever, or just Labrador, is a large breed of retriever-gun dog. The Labrador is the most popular breed of dog in Canada, the United Kingdom, and the United States.")